In [ ]:
# Based on https://github.com/computervisioneng/image-segmentation-yolov8/

### Check what classes (soorten)

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive')
import os
import re

ann_dir = "gdrive/My Drive/Stage/ClassificationData/Ready/10m/0bg/annotations"
ann_files = sorted([f for f in os.listdir(ann_dir) if f.endswith('.txt')])

def get_classes_sizes(ann_files):

  categories = []
  min_width = float('inf')
  min_height = float('inf')

  for ann_file in ann_files:

        # Get annotations: bounding box, segmentation (mask) and label
          with open(os.path.join(ann_dir, ann_file), "r") as file:
              annotations = file.readlines()

              # Get category
              for annotation in annotations:
                  # Process each line in the file
                  if annotation.startswith('Original label'):

                      # Extract category
                      match = re.search(r'Original label for object \d+ : "(.*?)"', annotation)
                      if match:
                          label = match.group(1)
                          categories.append(label)

              # Join lines into a single string
              annotations_str = ''.join(annotations)

              # Use regular expressions to extract width and height
              width_match = re.search(r'Image size \(X x Y\) : (\d+) x (\d+)', annotations_str)
              if width_match:
                  width = int(width_match.group(1))
                  height = int(width_match.group(2))

                  # Update the minimum width and height if necessary
                  min_width = min(min_width, width)
                  min_height = min(min_height, height)

  # Return categories and sizes
  return categories, min_width, min_height

categories, min_width, min_height = get_classes_sizes(ann_files)
soorten = sorted(list(set(categories)))
print("Dataset contains:", soorten)
print("Minimum width and heigth:", min_width, "x", min_height)

### Create YOLO labels
see https://docs.ultralytics.com/datasets/segment/#ultralytics-yolo-format

In [ ]:
import os
import cv2
import re

from google.colab import drive
drive.mount('/content/gdrive')

# Define the class labels (soorten)
class_labels = ['anders', 'dwergmeeuw', 'grote stern', 'kluut', 'kokmeeuw', 'visdief', 'zwartkopmeeuw']

mask_dir = "/content/gdrive/MyDrive/Stage/ClassificationData/Ready/5m/0bg/masks"
annotation_dir = "/content/gdrive/MyDrive/Stage/ClassificationData/Ready/5m/0bg/annotations"
output_dir = "/content/gdrive/MyDrive/Stage/ClassificationData/Ready/5m/0bg/labels"

for annotation_file in sorted(os.listdir(annotation_dir)):
    if annotation_file.endswith('.txt'):
        annotation_path = os.path.join(annotation_dir, annotation_file)
        image_path = annotation_path.replace("_annotation", "_mask").replace(".txt", ".png")

        # Read class labels and bounding boxes from annotation file
        with open(annotation_path, 'r') as ann_file:
            ann_content = ann_file.read()

            # Use regex to find the number of objects
            match = re.search(r'Objects with ground truth : (\d+)', ann_content)
            if match:
                num_objects = int(match.group(1))
            else:
                num_objects = 0

            # Initialize lists to store polygons and class indices for all objects
            all_polygons = []
            all_class_indices = []

            # Process each object in the annotation file
            for i in range(num_objects):
                original_label_match = re.search(r'Original label for object {} : "(.*?)"'.format(i + 1), ann_content)
                bbox_coords_match = re.search(r'Bounding box for object {} : \(Xmin, Ymin\) - \(Xmax, Ymax\) : \((\d+), (\d+)\) - \((\d+), (\d+)\)'.format(i + 1), ann_content)
                # mask_path_match = re.search(r'Pixel mask for object {} : "(.*?)"'.format(i + 1), ann_content)

                if original_label_match and bbox_coords_match: # and mask_path_match:
                    original_label = original_label_match.group(1)
                    bbox_coords = map(int, bbox_coords_match.groups())
                    mask_name = annotation_file.replace("annotation", "mask")
                    mask_path = os.path.join(mask_dir, mask_name[:-4]+".png")

                    class_index = class_labels.index(original_label)

                    # Load the binary mask and get its contours
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

                    H, W = mask.shape
                    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    # Convert the contours to polygons
                    all_polygons_per_object = []
                    cnt = contours[i] # only use the contour of the current object
                    if cv2.contourArea(cnt) > 0:
                        polygon = []
                        for point in cnt:
                            x, y = point[0]
                            polygon.append(x / W)
                            polygon.append(y / H)
                        all_polygons_per_object.append(polygon)

                    # Append polygons and class index to the lists
                    all_polygons.extend(all_polygons_per_object)
                    test_polygons = all_polygons
                    all_class_indices.extend([class_index] * len(all_polygons_per_object))

            # Print all polygons with corresponding class indices to a single YOLO label file
            img_name = annotation_file.replace("annotation", "img") # label needs to have the same name as the img
            with open(os.path.join(output_dir, '{}.txt'.format(img_name[:-4])), 'w') as f:
                for class_index, polygon in zip(all_class_indices, all_polygons):
                    line = '{} {}'.format(class_index, ' '.join(map(str, polygon)))
                    f.write('{}\n'.format(line))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### See if length of files in dataset match

In [ ]:
root_path = "gdrive/My Drive/Stage/ClassificationData/Ready"

# See if number of files match
print(len(os.listdir(os.path.join(root_path, "5m", str(0) + "bg", "images"))), "images",
      len(os.listdir(os.path.join(root_path, "5m", str(0) + "bg", "masks"))), "masks",
      len(os.listdir(os.path.join(root_path, "5m", str(0) + "bg", "annotations"))), "annotations",
      len(os.listdir(os.path.join(root_path, "5m", str(0) + "bg", "labels"))), "labels",
      "in directory", os.path.join(root_path, "5m", str(0) + "bg"))

14221 images 14221 masks 14221 annotations 14221 labels in directory gdrive/My Drive/Stage/ClassificationData/Ready/5m/0bg


### Make config and split files

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive')

import os
import cv2

# To avoid error: NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install ultralytics
!pip install opencv-python

import os, re, random
import cv2
from ultralytics import YOLO
# from utils.datasets import *
import yaml
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pathlib import *


In [2]:
# See https://github.com/ultralytics/yolov5/issues/1579

root_path = "gdrive/My Drive/Stage/ClassificationData/Ready"
IMG_FORMATS = 'bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm'  # include image suffixes

def img2label_paths(img_paths):
    # Define label paths as a function of image paths
    sa, sb = f'{os.sep}images{os.sep}', f'{os.sep}labels{os.sep}'  # /images/, /labels/ substrings
    return [sb.join(x.rsplit(sa, 1)).rsplit('.', 1)[0] + '.txt' for x in img_paths]

def autosplit(path, weights, annotated_only=False):
    """ Autosplit a dataset into train/val/test splits and save path/autosplit_*.txt files
    Usage: from utils.dataloaders import *; autosplit()
    Arguments
        path:            Path to images directory
                        --> assumes that the corresponding labels is in the same parent directory /labels/
        weights:         Train, val, test weights (list, tuple)
        annotated_only:  Only use images with an annotated txt file
    """
    path = Path(path)  # images dir
    files = sorted(x for x in path.rglob('*.*') if x.suffix[1:].lower() in IMG_FORMATS)  # image files only
    n = len(files)  # number of files
    random.seed(0)  # for reproducibility
    indices = random.choices([0, 1, 2], weights=weights, k=n)  # assign each image to a split

    txt = ['autosplit_train.txt', 'autosplit_val.txt', ]  # 2 txt files
    for x in txt:
        if (path.parent / x).exists():
            (path.parent / x).unlink()  # remove existing

    print(f'Autosplitting images from {path}' + ', using *.txt labeled images only' * annotated_only)
    for i, img in zip(indices, files):
        if not annotated_only or Path(img2label_paths([str(img)])[0]).exists():  # check label
            with open(path.parent / txt[i], 'a') as f:
                f.write(f'./{img.relative_to(path.parent).as_posix()}' + '\n')  # add image to txt file

autosplit(os.path.join(root_path, "5m", str(0) + "bg", "images"), weights=(0.9, 0.1, 0.0))

class_labels = ['anders', 'dwergmeeuw', 'grote stern', 'kluut', 'kokmeeuw', 'visdief', 'zwartkopmeeuw'] # = soorten

data = {
    'path': os.path.join(root_path, "5m", str(0) + "bg"), # root path, start at home directory
    'train': "autosplit_train.txt",
    'val': "autosplit_val.txt",
    'nc': len(class_labels), # number of classes
    'names': class_labels
}

with open(os.path.join(root_path, "5m", str(0) + "bg", "config.yaml"), 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

Autosplitting images from gdrive/My Drive/Stage/ClassificationData/Ready/5m/0bg/images
